In [0]:
# required library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [10]:

df = pd.read_csv('https://raw.githubusercontent.com/PanthonImem/Content-Based-Lab-Notebook/master/userLog_201801_201802_for_participants.csv', delimiter = ';', error_bad_lines = False, low_memory = False)
df.head(3)

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018,1,1,0


In [11]:
test_users = pd.read_csv('https://raw.githubusercontent.com/PanthonImem/KNN-Recommedation/master/testing_users.csv', delimiter = ';', error_bad_lines = False, low_memory = False)
test_users.head(3)

,userCode
0,003366c6-0cd9-48e7-d134-2051d7360c2d
1,003b2f18-60b4-37be-c63d-0caead1afa97
2,003d571e-9de7-6de3-2cb6-2ae190cc7beb


In [12]:
df_train = df[df['userCode'].isin(set(test_users['userCode']))]
df_train['interacted'] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
# Convert the dataframe into a matrix using pivot() and set index of df_train for fast search 
df_train_indexed = df_train[['userCode', 'project_id', 'interacted']].drop_duplicates()
df_train_pivot = (df_train_indexed.pivot(index = 'userCode', columns = 'project_id', values = 'interacted')
                                  .fillna(0))
df_train_pivot.head(3)

project_id,4,44,45,73,115,133,189,192,219,223,...,9483,9484,9486,9489,9491,9492,9493,9494,9496,9499
userCode,,,,,,,,,,,,,,,,,,,,,
0015e266-4175-bd2b-f3c2-945e61ad0cb6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00306a7d-e425-653c-03cb-ce4a3a478dcb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0032f4d6-6391-1f2c-14f6-73212fa9c0f8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_train_matrix = df_train_pivot.values
df_train_matrix.shape

(7408, 3477)

In [15]:
df_train_indexed = df_train_indexed.set_index('userCode')
df_train_indexed.head(3)

,project_id,interacted
userCode,,
57c6ecfb-184d-cc4e-973c-31ac6a7792cc,6511,1
57c6ecfb-184d-cc4e-973c-31ac6a7792cc,2332,1
57c6ecfb-184d-cc4e-973c-31ac6a7792cc,3004,1


## Implementing matrix factorisation for recommender system
For more infomation, http://nimfa.biolab.si/

In [16]:
!pip3 install nimfa
import nimfa

    100% |████████████████████████████████| 5.7MB 5.1MB/s 
  Running setup.py bdist_wheel for nimfa ... - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/63/b7/fd/858ba29683d798fd121b1df0d672943449e64e2ae94fd42af3
Successfully built nimfa


In [0]:
snmf = nimfa.Snmf(df_train_matrix, seed="random_vcol", rank=70, max_iter=12, rel_error=1e-5)
snmf_fit = snmf()

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:605: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  K = np.linalg.lstsq(CtC, CtA)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]
/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input m

/usr/local/lib/python3.6/dist-packages/nimfa/methods/factorization/snmf.py:622: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  sol = np.linalg.lstsq(A, B)[0]


In [0]:
W = snmf_fit.basis()
H = snmf_fit.coef()

In [0]:
result = (W*H)
result.shape

(7408, 3477)

In [0]:
colnames = df_train_pivot.columns
user_list = list(df_train_pivot.index)

In [0]:
result_df = pd.DataFrame(result)
result_df['userCode'] = user_list
result_df = result_df.set_index(['userCode'])
result_df.columns = colnames
result_df.head(3)

project_id,4,44,45,73,115,133,189,192,219,223,...,9483,9484,9486,9489,9491,9492,9493,9494,9496,9499
userCode,,,,,,,,,,,,,,,,,,,,,
0015e266-4175-bd2b-f3c2-945e61ad0cb6,9.602127e-07,0.000022,0.000072,1.768465e-16,0.000001,2.140914e-17,0.000078,0.000036,0.000106,0.000117,...,1.242429e-15,1.147677e-04,7.290279e-18,9.137139e-10,2.065618e-05,3.023071e-07,0.000001,8.903326e-18,1.620090e-17,3.136501e-06
00306a7d-e425-653c-03cb-ce4a3a478dcb,2.001155e-04,0.000645,0.000115,4.167887e-03,0.000428,1.254775e-04,0.000799,0.000097,0.000215,0.000023,...,1.263010e-15,6.809788e-05,1.078395e-05,2.430650e-17,3.696481e-03,5.409869e-05,0.000156,1.387250e-04,1.076919e-08,2.515312e-17
0032f4d6-6391-1f2c-14f6-73212fa9c0f8,4.928295e-03,0.002421,0.000004,2.779951e-04,0.000383,3.133308e-05,0.000107,0.010418,0.000431,0.000003,...,1.255588e-15,3.493577e-16,2.044994e-17,1.249953e-07,9.917759e-17,4.281519e-04,0.000020,2.206298e-17,2.936055e-17,1.773144e-17


#### Recommended items

In [0]:
def get_item_interacted(df, user_id):
        # to ignore interacted itemset when recommend
    """
    Args:
    - userId = user id
    Return:
    - set of interacted itemsets
    """
    interacted_projects = df[df.index == user_id][['project_id']]
    return set(interacted_projects['project_id'].values)

In [0]:
recommend_df = pd.DataFrame(columns = ['userCode', 'project_id'])
recommend_df

,userCode,project_id


In [0]:
for i in user_list:
    
    user_id = i
    items_to_ignore = get_item_interacted(df_train_indexed, user_id)
    pred_df = result_df[result_df.index == user_id].reset_index()
    pred_df = pd.melt(pred_df, id_vars=["userCode"], var_name="project_id", value_name="rating")
    pred_df = (pred_df[~pred_df['project_id'].isin(items_to_ignore)]
                                             .sort_values(['rating'], ascending=False)
                                             .head(7))
    pred_df = pred_df[['userCode', 'project_id']]
    recommend_df = pd.concat([recommend_df, pred_df])

In [0]:
recommend_df.head(10)

,userCode,project_id
1216,0015e266-4175-bd2b-f3c2-945e61ad0cb6,5396
109,0015e266-4175-bd2b-f3c2-945e61ad0cb6,2028
1182,0015e266-4175-bd2b-f3c2-945e61ad0cb6,5323
2319,0015e266-4175-bd2b-f3c2-945e61ad0cb6,7694
2455,0015e266-4175-bd2b-f3c2-945e61ad0cb6,8015
2953,0015e266-4175-bd2b-f3c2-945e61ad0cb6,8734
1765,0015e266-4175-bd2b-f3c2-945e61ad0cb6,6506
3121,00306a7d-e425-653c-03cb-ce4a3a478dcb,8997
1722,00306a7d-e425-653c-03cb-ce4a3a478dcb,6443
1273,00306a7d-e425-653c-03cb-ce4a3a478dcb,5500
